# Milestone 3

In this milestone, we will be setting up our spark cluster and developing our machine learning model. These were the tasks that we did:
 
(1) Setup EMR cluster  
(2) Develop ML model using scikit-learn  
(3) Obtain the best hyperparameters using spark's MLlib

## 1. Setup EMR Cluster

![](img/emr_cluster.png)

## 2. Setup browser, Jupyter environment and connect to Master node

![](img/jupyterhub.png)

## 3. Develop ML model using scikit-learn

Look at [Milestone 3 Task 3 file](Milestone3-Task3.ipynb) to view our work.

## 4. Obtain best hyperparameter

Look at [Milestone 3 Task 4](Milestone3-Task4.ipynb) file to view our work.

## 5. Deployment of model

![](img/s3_joblib.png)